In [1]:
import torch.nn as nn
import torch
from torch.autograd import Variable
%run nets.ipynb

In [2]:
def getTestAccAndLoss(modelpath, modelName, datapath, batch_size = 16):
    #this works like the eval part in my train function
    
    device = torch.device("cuda:0") if torch.cuda.is_available() else "cpu"
    #get model
    model = torch.load(modelpath)
    #loadData
    dataTest = torch.load(datapath + '/data_test.pt').float()
    labelsTest = torch.load(datapath + '/labels_test.pt').long()
    poseTest = torch.load(datapath + '/data_test_key.pt').float()
    poseTest1 = torch.load(datapath + '/data_test_key_1.pt').float()
    with torch.no_grad():
        #prepare eval
        error = nn.CrossEntropyLoss()
        model.to(device)
        model.eval()
        correct1 = 0
        total = 0
        correct2 = 0
        correct3 = 0
        totalLoss = 0
        weightedHier = [1/3,1/3,1/3]
        num_Batches_test = int(dataTest.shape[0] / batch_size)
        if dataTest.shape[0] % batch_size != 0:
            num_Batches_test += 1
        for i in range(num_Batches_test):
            #get the correct slice of the data
            cutBatch = (i+1)*batch_size
            if i == num_Batches_test - 1:
                cutBatch = None  
            #get the correct data for model
            labels = labelsTest[i*batch_size:cutBatch]
            if modelName != "pose":
                images = dataTest[i*batch_size:cutBatch]
            if modelName == "combined1" or modelName == "pose": 
                keysLabel = poseTest[i*batch_size:cutBatch]
            elif modelName == "combined2":
                keysLabel = poseTest1[i*batch_size:cutBatch]

            if modelName == "combined1" or modelName == "pose":
                keysLabel = Variable(keysLabel.view(keysLabel.shape[0],51))
            if modelName != "pose":
                test = Variable(images.view(images.shape[0],100,100,3))
                test = torch.transpose(test, 1, 3)
                test = torch.transpose(test, 2, 3)
            if modelName == "combined2":
                keysLabel = Variable(keysLabel.view(keysLabel.shape[0],1,100,100))
                test = torch.cat((test, keysLabel), dim=1)
            labels1 = Variable(labels[:,0])
            labels2 = Variable(labels[:,1])
            labels3 = Variable(labels[:,2])

            if modelName != "pose":
                test = test.to(device).to(torch.float)
            if modelName != "cnn" and modelName != "hier":
                keysLabel = keysLabel.to(device).to(torch.float)
            labels1, labels2, labels3 = (
                labels1.to(device),
                labels2.to(device),
                labels3.to(device),
            )


            #compute accuracies, loss and metrices
            if modelName == "cnn":
                outputs = model(test)
                loss = error(outputs, labels3)
            elif modelName == "pose":
                outputs = model(keysLabel)
                loss = error(outputs, labels3)
            else:
                if modelName == "combined1":
                    a,b,outputs = model(test,keysLabel)
                else:
                    a,b,outputs = model(test)
                predicted1 = torch.max(a.data, 1)[1]
                predicted2 = torch.max(b.data, 1)[1]
                correct1 += (predicted1 == labels1).sum()
                correct2 += (predicted2 == labels2).sum()
                loss1 = error(a, labels1)
                loss2 = error(b, labels2)
                loss3 = error(outputs, labels3)
                loss = weightedHier[0]*loss1 + weightedHier[1]*loss2 + weightedHier[2]*loss3
            predicted3 = torch.max(outputs.data, 1)[1]  
            total += len(labels3)
            correct3 += (predicted3 == labels3).sum()
            totalLoss += loss.sum()

        if modelName != "cnn" and modelName != "pose":
            accuracy1 = 100 * correct1 / float(total)
            accuracy2 = 100 * correct2 / float(total)
        accuracy3 = 100 * correct3 / float(total)
        epochLoss = totalLoss / float(total)


        if modelName != "cnn" and modelName != "pose":
            print(accuracy3,epochLoss)
        else:
            print(accuracy3,epochLoss)

In [3]:
getTestAccAndLoss("./bestModels/cnn_0.003_0.040", modelName = "cnn", datapath = "./AllData")
getTestAccAndLoss("./bestModels/hier_0.003_0.040", modelName = "hier", datapath = "./AllData")
getTestAccAndLoss("./bestModels/combined1_0.003_0.040", modelName = "combined1", datapath = "./AllData")
getTestAccAndLoss("./bestModels/combined2_0.003_0.040", modelName = "combined2", datapath = "./AllData")

getTestAccAndLoss("./bestModels/dense_hier_0.003_0.040", modelName = "hier", datapath = "./AllData")
getTestAccAndLoss("./bestModels/dense_combined1_0.003_0.0100", modelName = "combined1", datapath = "./AllData")
getTestAccAndLoss("./bestModels/dense_0.003_0.040", modelName = "cnn", datapath = "./AllData")
getTestAccAndLoss("./bestModels/dense_combined2_0.003_0.040", modelName = "combined2", datapath = "./AllData")

getTestAccAndLoss("./bestModels/pose_0.0001_0.0100", modelName = "pose", datapath = "./AllData")

tensor(40.0073, device='cuda:0') tensor(0.2550, device='cuda:0')
tensor(41.2779, device='cuda:0') tensor(0.1440, device='cuda:0')
tensor(38.6459, device='cuda:0') tensor(0.1518, device='cuda:0')
tensor(41.5683, device='cuda:0') tensor(0.1514, device='cuda:0')
tensor(3.1585, device='cuda:0') tensor(0.1966, device='cuda:0')
tensor(29.3883, device='cuda:0') tensor(0.1502, device='cuda:0')
tensor(2.7773, device='cuda:0') tensor(0.2765, device='cuda:0')
tensor(3.3400, device='cuda:0') tensor(0.1821, device='cuda:0')
tensor(31.5121, device='cuda:0') tensor(0.2008, device='cuda:0')


In [2]:
def printNumPara():
    model = myModel("cnn",dropout = 0.2)
    print('CNN: {}'.format(sum([p.data.nelement() for p in model.parameters()])))
    
    model = myModel("hier",dropout = 0.2)
    print('hier: {}'.format(sum([p.data.nelement() for p in model.parameters()])))
    
    model = myModel("combined1",dropout = 0.2)
    print('combined1: {}'.format(sum([p.data.nelement() for p in model.parameters()])))
    
    model = myModel("combined2",dropout = 0.2)
    print('combined2: {}'.format(sum([p.data.nelement() for p in model.parameters()])))
    
    model = Net_pose()
    print('pose: {}'.format(sum([p.data.nelement() for p in model.parameters()])))
    
    model = DenseNet3(82, "cnn", growth_rate=6,
                 reduction=0.5, bottleneck=True, dropRate=0.2)
    print('dense: {}'.format(sum([p.data.nelement() for p in model.parameters()])))
    
    model = DenseNet3(82, "hier", growth_rate=6,
                 reduction=0.5, bottleneck=True, dropRate=0.2)
    print('dense_hier: {}'.format(sum([p.data.nelement() for p in model.parameters()])))
    
    model = DenseNet3(82, "combined1", growth_rate=6,
                 reduction=0.5, bottleneck=True, dropRate=0.2)
    print('dense_combined1: {}'.format(sum([p.data.nelement() for p in model.parameters()])))
    
    model = DenseNet3(82, "combined2", growth_rate=6,
                 reduction=0.5, bottleneck=True, dropRate=0.2)
    print('dense_combined2: {}'.format(sum([p.data.nelement() for p in model.parameters()])))
    return 0

printNumPara()

CNN: 497806
hier: 1639464
combined1: 1771464
combined2: 1639614
pose: 190402
dense: 53302
dense_hier: 224838
dense_combined1: 361152
dense_combined2: 224946


0